# Generation of mexican municipalities names


Mexico has over 2400 municipalities with names where the root of their names come from nahuatl (Aztec's language), spanish, Maya, and many more regional languages. And because of that Mexico has a very rich diversity when it comes to names. 

This notebook tries to use a RNN to learn and generate names that seems from mexican municipalities names.

I'm using a [gist](https://gist.github.com/karpathy/d4dee566867f8291f086) from 
Andrej karpathy. But I'm changing the corpus to be the mexican municipalities names and try it to adjust it to get the best results I can with this corpus

### Libraries

In [1]:
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License
"""
import numpy as np
from random import uniform

### Loading corpus
Loading corpus from "municipios.txt" that can be found [here](https://raw.githubusercontent.com/RicardoHE97/RN-Unison/master/modelo_de_generacion_de_texto_con_redes_recurrentes/municipios.txt)

In [3]:
# data I/O
data = open('municipios.txt', 'r').read()
chars = list(set(data)) # Obtain the unique character used
data_size, vocab_size = len(data), len(chars) #size of the dataset y size of the unique characters
print('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

data has 35530 characters, 68 unique.


### Lost function 

In [4]:
def lossFun(inputs, targets, hprev):
    """
    Funcion de perdida
    
    Parámetros
    ----------
    inputs: número de entradas de la red
    targets: número de 
    hprev: 
    
    inputs,targets are both list of integers.
    hprev is Hx1 array of initial hidden state
    returns the loss, gradients on model parameters, and last hidden state
    """
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0
    # forward pass
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
        ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
        loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
    # backward pass: compute gradients going backwards
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(Why.T, dy) + dhnext # backprop into h
        dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(Whh.T, dhraw)
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

### Samples a sequence for the model

In [5]:
def sample(h, seed_ix, n):
    """ 
    sample a sequence of integers from the model 
    h is memory state, seed_ix is seed letter for first time step
    """
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = []
    for t in range(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    return ixes

### initialize parameters

In [14]:
tol = 18

# hyperparameters
hidden_size = 24 # size of hidden layer of neurons
seq_length = 12 # number of steps to unroll the RNN for
learning_rate = 0.07

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0

### Model training and test 

In [15]:
while True:
    # prepare inputs (we're sweeping from left to right in steps seq_length long)
    if p+seq_length+1 >= len(data) or n == 0: 
        hprev = np.zeros((hidden_size,1)) # reset RNN memory
        p = 0 # go from start of data
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

    # sample from the model now and then
    if n % hidden_size == 0:
        sample_ix = sample(hprev, inputs[0], 200)
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print('----\n %s \n----' % (txt, ))

    # forward seq_length characters through the net and fetch gradient
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    if n % hidden_size == 0:
        print('iter %d, loss: %f' % (n, smooth_loss)) # print progress

    # perform parameter update with Adagrad
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

    p += seq_length # move data pointer
    n += 1 # iteration counter 
    if smooth_loss < tol:
        break

----
 qH-Á
AáüúqUBkélÑ KíyaKF66jLzIDLmEJaVNjfKlcnh
SNlq F
nHAuáCX62
LjTyOBsÑ,TiCL00BsZrgL,lqxiCXsüOvxfFcNEobsimDnnPGF0MámlBa-u
R0eBIül-NñZaó.YúzVBRÑá
kYO
AijxeTFLú b
aVcxáe8o8Fé-LLBJédFü
KsÑRSoVüG0dhZAi
dV0 
----
iter 0, loss: 50.634093
----
 acácaca
edAcacAgayálaePcac
écctauorAñaccco
A
AA
acacyczeaenas

AnanlletenaalacAco
ucaAsce rzucaa
Acac
ecepínoce aactAceeelrcaca
ácaAtaAAc
A
k
ocajstntozáác acccúAóetatnsapAcAca
Aca
lca-aceasy
acajAcac 
----
iter 24, loss: 50.358085
----
 
Aa
e xAca

Acuectaaalla eg
Alua
eluasItnal
A ua
oáetahrluaeepeáázta JHtuAcuanCltala
Alodotáz goAnuaJhculhlo eellaáacl
e
Ag
nouohuerlee
ocueBñlt pc
 nuatA ElulznnCA
AhletFGzAcu
oheoa
AcuUpAgl
AtA
icun 
----
iter 48, loss: 49.926106
----
 AldralolbllH Jueddacmy
An elvpomalueo iecarlleaontldirleolhhnla
Aluau


Alabddepao
Ahl
el
Ale
Asa ortl
AlUsllueeV
AVzydyed J ldlA olarayseg
Ahe
Axlryaldeollgpo
Anlaldtalj ñAgl
Allo
Audammatile  Galn
A 
----
iter 72, loss: 49.531034
----
 nrt SísatoPoede x
Acasrrot
Ajat
Amo. plncuy

----
 
Ietlooco
dutla
dao R.s
Ixspa
Guluacaca
Ixcopadtlgan
Boyqpo depel depo tlán dalán dahcaml deco
Calsaic
Ddt. zelaxte da da Ilma
Alapapárla
Ixcala
do
Iltahiluitira Pfe
Ixteuciapamioa 
Ixtleu dezo
Ipánob 
----
iter 840, loss: 38.172799
----
 genl éMan
Huttzlmidaz DlleLa
Itape  Jolqodare Jalosse
Jamarcapogi
Iraste
íeTden
Jeroerda JiájAran
Jeyo titomexladc
Hexpanseca
Jraco
Jád Jabar
Jxnlal
Jayel
FrdenJex Bapes
Jalga
Jinpa
Iolpamhas Ixtácoze 
----
iter 864, loss: 37.926888
----
 Jau
Juaxtanitípaguc Vo Juj
Juas Mic
Jerazo Jei g.ñ
Fatttán
Héciac
JoltlaÁyiez
Jéjanal 
Jupc Mas
Juo
Jopua
JoCJa
Jo Oeután
Jes
Buaqoscaz
Goma
Jenlo
Joy
Cayetía
Jenhlopán
JelLla
J
Huatavn
Jeré
dannDe Oe 
----
iter 888, loss: 37.672209
----
 Ti C. R,gBal Hayo
Irumclopc
Erjjla. Chuicdú Zamuripo
Lias Je JeTle
Ií Himua di Ja Mvire pecra
JAposyolo
Pelte Junldar
Hintzón
Jas
oncan
Iyarila
Moldico
Jeleán
Jolhuep Zapén
Jepaz
Cupsto
Loríagven d.L
 
----
iter 912, loss: 37.482010
----
 c Pe Jaypi Ja Sere
La ñlnepe Oía
Ma,dl

----
 an Juan Jeroctila
San Caritlho Ttexcos Juan Juan Catime
Sas Juan Juinatatlán Alla Jonte
San Guec Juacuma
San Flis dan
San Áuan uicuan Juan Ruarda
San Juanunaco
Sanlo
San Atas huan Inan Ocostozoto Humo 
----
iter 1680, loss: 30.813848
----
 n Juin dua
Sin Jeisolede GaAtonc Cuetilán Qnantayotigahuapié Juan Fcan Tachen JurBpan Emalbos Lo JuaZ o man
Sanoco
San Juanczen
San Juan Jucalí
Saxcacatemé Tuilas Ralas
Ste Shfavile
San Jua
Sah Cuin A 
----
iter 1704, loss: 30.586813
----
 Sanl
San Dvilan Asontaziitilexco Cuan Ogosciledo Juan Juan San Fétezopíces
San Suan San PQunap
San Cuueraco
San Juanteé usulanni íñilcosgiatlovo
Sanupetóntlzaltooco
Sancuon Juan Estícióntilaüdes Juaxd 
----
iter 1728, loss: 30.367755
----
 ahdes
San Escor LuGar LecBan KurgosPo
San Unan Cuilitrán Yíirlán
San Tanio Morimos
San mqan Huacués
San Tuan Najivipa
Són Juan Róan Man Luaj ñélalco
San Juantipig Juacosm
San Ixtstede
San FuacGin La d 
----
iter 1752, loss: 30.150705
----
 e
Man Amvilan Ruua dostes
San Juel

----
 clá
Seltima
Teresco
Terínal
Tocabuos Tío Inoscala
Tepemoóc
Taraso
Tepenca
Sanda Yecaxto
Gancomomergo de Zeracazua
Teperach.
Tepada
Temalgo Terosgo
Pemantiayo
Tebapengi
Temochimhosjogual Xepanco Tovech 
----
iter 2568, loss: 25.435881
----
 pipo
Tepedel
Maltope da
Tecaila do depildo
Tejextla To Macolánte Tepitla
Telago
Tepetetates
Temema deCtan
Telán
Leacuila
Tenquela
Tepadar
Sante TesóI Paánco
Tecopecho
TeNotliguitta
Tehus
Tepián Teken
 
----
iter 2592, loss: 25.358426
----
 oco
Teguali
Temapaltecino Teyeloto Mamanca
Tecococo
TeraliAl dides
Tetaltago
Tea
Tepego
Tóxtlrextas
Tezuanoga Anzo
Tetáz
Sacocalla
Teérhuacrel
Jueáncotel
Textlán
Teúscante Intepes delcigcantla
Tecahua 
----
iter 2616, loss: 25.314689
----
 itlilzcabtiagda
Tería
Temasdeca
Tlángo
Tahuatepipa des di Dlandión
TepepidrMfac dekgs
TexmBotla Noza
Toxcocuiatlán
Tinuca
Tixtlo IlárU
Tixxitta Vosgía
Toza queltían
Tinán
Dapistaanc
Teáguado de
Temama 
----
iter 2640, loss: 25.332814
----
 ca
Tíinco
Tayiruixl de da deca
Tll

----
 tijuato
Otucutellán
Cuanehuinta
Ololán
Cerra
Coragalos
Citetola
Coópin
Cotlantlasdes
Cocpana
Ciuríotz Coyopan
Chumlo
Colaltlzo
Cuhicula
Cuiruaro
Coteya Coapatlpes
Comarez
Catlánomatepatel
Cuaritatel d 
----
iter 3456, loss: 25.032117
----
 ilcaamapea
Cuacocuacaogo del Ciarón Cuiroreco
Cuqualán de Moyepna

Cuuumaquexcutlán del
Apatecyacos
Cuitáro
Cuénunlo
Cuatloconpizula
Terrexde Incualda Coyomjitl
Cuerrexto
Cánidrica deá
Cheaca
Cijrañán 
----
iter 3480, loss: 25.022950
----
 
demal dez
Cullán
Axtíi Jo
Frreyba
Cuialude
Dulatcada dedro de Hunulo de Meruár
Póle Geéxtas
Cithemaz
Cualúvila Jo GuinCúa
Zuhistlár
Cuicapepác
Caxatelo
Axtláa debgo
Cárarez Deñis
Cugupara
Cuzall
Zhua 
----
iter 3504, loss: 25.225445
----
 eco
Carantlo de de Zlán Codeár Les dede
Jululhi
de yezan
Elzorez
Gosunev Re Gumaca
Elmoncro
Canas Po Fiitlán
Elán drán
Elden
Almotlác
Sotza
Cuedi de Morcini de Guanatii
Erca demo
Bromana Moyisrde
Rurd 
----
iter 3528, loss: 25.292858
----
 pecapio
Etayonez
GuJonalvan
Alari


...

----
 olápac Tullota
Gedesisción
Huito
HualpanVillapa
Moxpe do Valvíd
Gajemox
Inahñola
Villa Grillisconel ela Heduz Criac
Putrtaro
Valostlán del Ríón Chereya
Aloma
El Catlahman des Juava
Conenes
Bantoyuapan 
----
iter 205224, loss: 22.092374
----
 
Oine
Gonimaro
Josuolapa
Aimatlána Villa
Lecórapa Aluscuico
Juquiconela
Letóbo
Bería Magdalan de Julatario
Hueloca
Ercocoapa
Mixtla
Itos
Cubena
Atiahúc
Cirantla
Sanixtzina
Apéruera Villa de de Santrén 
----
iter 205248, loss: 22.135213
----
 rialo Trisá
Valdo
Ixcánciniíco
Guedrar
Banoa Chiacuac
Matlán
Cosco
Frincimuzl de Maríado
Juen Amunco
Xintealá
Lanaco Esxichanco
delcos
Lon Agrraz. Mantaper
Cagda Minoruer
Iruín de Trascitlán
Coyucaman 
----
iter 205272, loss: 22.147355
----
 án
Felariego
Ziedad de Gerrezo Blanger
Alárcistepic
Hemuón
Minutlán
Marasin de de lón Cuazazantla
Lo Crue Larinos
Juárez
Bauto
Tecia
Zientepen
El L. Cama
Chidalle
Mozanata Mamezo
Génes de bengaltenjo
 
----
iter 205296, loss: 22.124415
----
 zalis
Ixtamaltás
La Marere

----
 ocolZán Madolán
San Magero
Morinaz
San Perro Nodalo
San Merén Lugón
Sárero
Savañán
San Juan Guila de larradas
Zahuhulás
Lnchigomo
Sactonga
San Pargonja
San Anganocisfos Carra
San Juan Quesmano
San Sua 
----
iter 206112, loss: 20.258818
----
 n Satristl Ban Tranastechichima
San Marrelos
San DeIndano Narque Minos
San Pedrde
San Plío Cajio Lucoss Niriago Heyas
San Maguez Frris. Monteoxán
San Páredango de Soa Madaro
Virciaste
Santo Foría de M 
----
iter 206136, loss: 20.167609
----
 an Cajicuen
Savo
San Migaro
Sostampo
San Pedro Yoluz
Ayotlago
Quitrenango Chematlotopec
San Pedrdes Momelzencín Zaba
San Marcos
San Méría Tepac
Tztante
San Magoro Tlacopeo Coyapac
Santa Mazté de Dolis 
----
iter 206160, loss: 20.027204
----
  Rome
San Agrande
San Mikutlán de Mercis
San Antocochebo Hidango
Ajos
San Pedro Hextitla
Santa Ixtic Chón de Carzan Aquinicho
San Puen de Dotza
San Fereo
Sas Jianixten
San Estaya Tinajas
San Perraro I 
----
iter 206184, loss: 19.912555
----
  Isé Cenelvón
San Juen Si


----
 lhu
Tlápac
Telcol
Cullantepe
Tlaqueca
Tlahaguac
Tláxtoman
Tepaxpar
Tojichiltepec
Santo yoregdo
Tesuacua
Caños Concaltialó
Tima
Ahuecamado
Dixtepec
Tenaldo
Xanis P. Brano Cadalten Tetacahuatectán
Ahult 
----
iter 207000, loss: 19.249048
----
 a Tonjo de Dosisto
Tetzilantenan Medaz
Omocao
Mahualánto
Tantiego
Ixicolá
Ticouteper
Santa Tlán de Lulisgo Tunaltiantiago
Tetlani
Tocotos
Pomango Olacuantepec
Otiaquilpones
Teuquatepetlo
Techilpatepos 
----
iter 207024, loss: 19.376718
----
 e Brandero
Tuenco Atorelas
Santa P
Santa María Tetlapa
Tusuohue
Tlalco
Tlal Grandotel Trueyata
Chimulapa
La Mamimúc
Romegal
Saddo
Tasyatlán del Tigayuc
Yinetepac
Tanavoco
Teniso
Tomaro
Santamo
Telutan 
----
iter 207048, loss: 19.400512
----
 rías
Oyila
Tecolac
Santa
Textzanco
Tepao
Testien
Atlauaco
Temetepac
Caluca
Ucfirulachia
Tuerana
Santa Imézomeral
Santa Grdrerdo
Salaciz
Zanticandos
Tecobo
Toltepeca Tulión de Ánguma Joninaro
Tocuma
Ta 
----
iter 207072, loss: 19.388452
----
 zianca de Juen del Tlacual

----
 eSantipetlán
Cididerratis
Nancellla
Humos
Conalco Cabonalla
Lacamelte
Coampin
Bonéyo
Motamatipo
Cuzhapalco
yaotetzintapacay Huixtere
Caris dré Esnandimena
Coyaca
Cayápe
Jamorhuintepeb
Conemas
Halal Gu 
----
iter 207864, loss: 21.790823
----
 tollo
Ayapachitlán
Xoltianan
Zualdaro
Foralano
Cuacullitla
Coicablepren
Catejetlán
Chiación
Acaroco
Guatepec
Ahuitlixquimunas
Amancos
Bebrerris
Alaroser
Chizuzi
Coucumón
H. Mastilos
Otono
Ajonciac
Lat 
----
iter 207888, loss: 21.875570
----
 
Mordeo
Ecolrica
FJovepa
Bucomacuía
Janyaniema
Locuinto
Ecotepec
Hieaní
Bautimuela
Bamala
Cuatro
Cuaagaz
Aemextata
Coria de Ría
Biacarielo Heyec
Ganiscón Cruza
Elliagual
Cuagomán de Gombandado
Dontiog 
----
iter 207912, loss: 21.879873
----
 anóntepa
Gunerosé
Santa Grdetos
Locidad de Carhiachu
Coyomaquia de Guetla
Corteago
Tesatlán
iapucua
Pixocol del Arés
Felo Emumán
Masixtepec
Caalco decitepen
Ahuipanga
Calcotepec
Urán de Guarro
Cuáreca 
----
iter 207936, loss: 21.935392
----
 Coyomoy
Zapancapa
Alpepiba

----
 ez Camitlán
Sas Allo
Arapan
San Alvio
San Migaro Hoitemirco Lilo Magoro
Penír
Sa lacharciniz Olangal CauciateTla deSandrca
San Ano
Santa Alomán Ozo de -Dgosto
San Mixtepec
Santa Aníguós Tetosepen
Pelg 
----
iter 208728, loss: 21.991221
----
  Cranta Nala
Cuas
Sabarida
San Juan Cautlán de Etzan
Santa de Arantía
Maritla de la Solterca
Donimán de Alcoso
Frarroradro de Colozonos
San Puen DBárada
Dolil
San Bartel valióre de IsOtos Milaztepec
S 
----
iter 208752, loss: 21.897103
----
  lapegno Solixtequia de Jelóa
Sen Pedro Hosixca
San Pebaro didros Cabtez
Moría Tlapac Tixto Vel Repe Ocata Huírtecac
Santo Jecunac
San Bebrernez
Zixecol de Dosiste Ridó Larín de Vielll Tlaxtlago Grana 
----
iter 208776, loss: 21.854301
----
 sasisco Couringo
San Pedrés Cringuelás Man án Golera
Sal Flanfon
Pa Fradrrés Hebrechilá
Quixtepec
San JuFra
San Frinca Comitín Ozomachuz
San Juan Isbarde
Jachín
Maritlán Frehío
Jechitepec
San Felilco  
----
iter 208800, loss: 21.722336
----
 an Miguiaz Zúca
San Astana